## Reads covariates.xlsx and biomarkers.xlsx and flatten all variables related to a single patient.

In [1]:
import pandas as pd

In [3]:
xls = pd.ExcelFile('covariates.xlsx')
covariates = pd.read_excel(xls, 'Ark1')
covariates.describe

<bound method NDFrame.describe of      PatientID  Age  Sex (1=male, 2=female)  Smoker (1=yes, 2=no)  \
0            1   56                       1                     2   
1            3   32                       1                     2   
2            4   43                       2                     2   
3            5   25                       2                     2   
4            6   39                       1                     2   
..         ...  ...                     ...                   ...   
113        143   55                       1                     1   
114        149   46                       2                     2   
115        150   57                       1                     1   
116        151   25                       2                     2   
117        152   53                       1                     2   

     VAS-at-inclusion  Vas-12months  
0                 3.0           4.0  
1                 7.2           0.5  
2                 2.7  

In [41]:
biomarkers = pd.read_excel('biomarkers.xlsx', index_col=0)
biomarkers = biomarkers.reset_index()
biomarkers = biomarkers.rename(columns={'Biomarker': 'PatientID'})
biomarkers.describe

<bound method NDFrame.describe of         PatientID  IL-8  VEGF-A    OPG  TGF-beta-1  IL-6  CXCL9  CXCL1  IL-18  \
0      126-0weeks  7.63   11.51  10.20        8.83  3.52   6.16   9.45   7.91   
1      126-6weeks  7.12   11.59  10.41        8.87  3.89   6.12   9.06   7.92   
2      127-0weeks  6.93   10.92  10.30        6.59  2.73   6.14   7.31   7.95   
3      127-6weeks  7.16   11.58  10.39        8.61  2.60   6.35   8.61   7.94   
4    127-12months  6.87   11.13  10.25        7.44  3.92   6.15   8.79   7.94   
..            ...   ...     ...    ...         ...   ...    ...    ...    ...   
342    121-6weeks  7.21   12.89  10.69        8.90  3.14   5.97   8.00   8.17   
343  121-12months  6.35   11.01   9.96        6.70  2.55   4.68   7.34   7.76   
344    122-0weeks  7.86   12.80  10.73        9.30  2.93   6.73   9.60   8.25   
345    122-6weeks  8.52   13.04  10.81        8.79  2.94   6.73   9.64   8.19   
346    124-0weeks  6.51   11.38  10.07        7.78  3.29   5.97   7.17   8.

### Split df into three dataframes containing the three points (0weeks, 6weeks, 12months)

In [42]:
zero_weeks_biomarkers = biomarkers[biomarkers['PatientID'].str.contains("0week")].rename(columns={c: 'zero_weeks_'+c for c in biomarkers.columns if c not in ['PatientID']})
six_weeks_biomarkers = biomarkers[biomarkers['PatientID'].str.contains("6week")].rename(columns={c: 'siz_weeks_'+c for c in biomarkers.columns if c not in ['PatientID']})
twelve_months_biomarkers = biomarkers[biomarkers['PatientID'].str.contains("12months")].rename(columns={c: 'twelve_months_'+c for c in biomarkers.columns if c not in ['PatientID']})

twelve_months_biomarkers.head(3)

,PatientID,twelve_months_IL-8,twelve_months_VEGF-A,twelve_months_OPG,twelve_months_TGF-beta-1,twelve_months_IL-6,twelve_months_CXCL9,twelve_months_CXCL1,twelve_months_IL-18,twelve_months_CSF-1
4,127-12months,6.87,11.13,10.25,7.44,3.92,6.15,8.79,7.94,8.46
7,128-12months,6.47,11.05,10.14,6.45,4.65,8.00,8.18,8.71,8.56
10,129-12months,6.56,10.84,10.50,7.42,3.01,5.81,8.82,7.62,8.39


### add prefixes to all variables

In [43]:
zero_weeks_biomarkers['PatientID'] = zero_weeks_biomarkers['PatientID'].str.removesuffix('-0weeks').astype('int64')
six_weeks_biomarkers['PatientID'] = six_weeks_biomarkers['PatientID'].str.removesuffix('-6weeks').astype('int64')
twelve_months_biomarkers['PatientID'] = twelve_months_biomarkers['PatientID'].str.removesuffix('-12months').astype('int64')

twelve_months_biomarkers.head(3)

,PatientID,twelve_months_IL-8,twelve_months_VEGF-A,twelve_months_OPG,twelve_months_TGF-beta-1,twelve_months_IL-6,twelve_months_CXCL9,twelve_months_CXCL1,twelve_months_IL-18,twelve_months_CSF-1
4,127,6.87,11.13,10.25,7.44,3.92,6.15,8.79,7.94,8.46
7,128,6.47,11.05,10.14,6.45,4.65,8.00,8.18,8.71,8.56
10,129,6.56,10.84,10.50,7.42,3.01,5.81,8.82,7.62,8.39


### get count of unique patient IDs for covariates and biomarkers

In [44]:
covariates['PatientID'].nunique()

118

In [32]:
zero_weeks_biomarkers['PatientID'].nunique()

117

In [49]:
pd.concat([zero_weeks_biomarkers['PatientID'],covariates['PatientID']]).drop_duplicates(keep=False)

30    40
Name: PatientID, dtype: int64

In [34]:
six_weeks_biomarkers['PatientID'].nunique()

116

In [50]:
pd.concat([six_weeks_biomarkers['PatientID'],covariates['PatientID']]).drop_duplicates(keep=False)

26    36
38    49
Name: PatientID, dtype: int64

In [51]:
twelve_months_biomarkers['PatientID'].nunique()

114

In [52]:
pd.concat([twelve_months_biomarkers['PatientID'],covariates['PatientID']]).drop_duplicates(keep=False)

38     49
90    117
94    122
96    126
Name: PatientID, dtype: int64

<font color='red'>Incomplete data for some patients (36, 40, 49, 117, 122, 126)</font>

### double merge to merge the three df into one. Use outer join to account for incomplete patient data (MISSING VALUES NULL)

In [53]:
complete_biomarkers = pd.merge(pd.merge(zero_weeks_biomarkers, six_weeks_biomarkers, on='PatientID', how='outer'), twelve_months_biomarkers, on='PatientID', how='outer')
complete_biomarkers.head(3)

,PatientID,zero_weeks_IL-8,zero_weeks_VEGF-A,zero_weeks_OPG,zero_weeks_TGF-beta-1,zero_weeks_IL-6,zero_weeks_CXCL9,zero_weeks_CXCL1,zero_weeks_IL-18,zero_weeks_CSF-1,...,siz_weeks_CSF-1,twelve_months_IL-8,twelve_months_VEGF-A,twelve_months_OPG,twelve_months_TGF-beta-1,twelve_months_IL-6,twelve_months_CXCL9,twelve_months_CXCL1,twelve_months_IL-18,twelve_months_CSF-1
0,1,8.13,12.35,10.48,8.66,2.63,6.54,9.54,8.53,8.27,...,8.19,8.64,12.48,10.68,8.46,2.56,6.64,9.59,8.79,8.41
1,3,6.55,11.21,10.49,6.83,2.58,5.31,6.71,7.71,8.30,...,8.35,7.44,11.67,10.70,7.81,2.72,5.54,8.78,7.77,8.51
2,4,6.47,11.13,10.72,6.90,5.62,5.46,7.73,8.02,8.19,...,8.27,7.22,11.50,10.83,7.41,4.33,5.75,8.24,8.19,8.18


In [54]:
complete_biomarkers.describe

<bound method NDFrame.describe of      PatientID  zero_weeks_IL-8  zero_weeks_VEGF-A  zero_weeks_OPG  \
0            1             8.13              12.35           10.48   
1            3             6.55              11.21           10.49   
2            4             6.47              11.13           10.72   
3            5             6.41              11.15           10.60   
4            6             6.54              11.47           10.20   
..         ...              ...                ...             ...   
113        143             6.67              11.12           10.60   
114        149             8.58              12.90           11.30   
115        150             6.15              11.05           10.43   
116        151             7.98              11.89           10.79   
117        152             6.93              10.95           10.46   

     zero_weeks_TGF-beta-1  zero_weeks_IL-6  zero_weeks_CXCL9  \
0                     8.66             2.63              6.5

### test that complete_biomarkers and covariates are the same lenght and that they contain the same patient ids with no unique values in either

In [56]:
complete_biomarkers['PatientID'].nunique() == covariates['PatientID'].nunique() and len(pd.concat([complete_biomarkers['PatientID'],covariates['PatientID']]).drop_duplicates(keep=False)) == 0

True

### merge complete_biomarkers and covariates

In [58]:
covariates_biomarkers = pd.merge(covariates, complete_biomarkers, on='PatientID', how='outer')

### produce .csv

In [59]:
covariates_biomarkers.to_csv('covariates_biomarkers.csv', encoding='utf-8', index=True)